In [1]:
import requests
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

prenset = "出席"
absent = "缺席"
leave = "請假"
url = "https://script.googleusercontent.com/macros/echo?user_content_key=CS1xGtawxafbsz_VXLceDZ8YhZJ3WLRGo3cTurMdyLFpRky5qzeXoqZBrYuJ0ElreSkNapo1i9TRXgBoQpou4xhHPHjfNUj4m5_BxDlH2jW0nuo2oDemN9CCS2h10ox_1xSncGQajx_ryfhECjZEnNGpsruz596piGbFJMaeWeZjFnhN0a9d4HwmOCyefi3ex5PPcpNsPlxGCUpGq7rwIA3gVkC6Pkbl8Md7CuyY-jH4-8QW02051A&lib=McNoStEX3P6w1EymfWUjOXqNKe58B53cM"

In [2]:
def getData(url: str):
    try:
        response =  requests.get(url)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
    return None

In [4]:
def getDf_from_google_sheet_json(resp_json):
    df = pd.DataFrame(resp_json['data'][1:], columns=resp_json['data'][0])
    strings = df.columns
    pattern = r"\[(.*?)\]"
    result = []
    for string in strings:
        matches = re.findall(pattern, string)
        result.extend(matches)
    df.columns = list(df.columns[:2]) + result
    df['date'] = pd.to_datetime(df['日期']).dt.strftime('%Y-%m-%d')
    cols = df.columns.tolist()[2:]
    cols = [cols[-1]] + cols[:-1]
    output_df = df[cols].copy()
    return output_df

In [5]:
def getDf_present_rate(df, prenset, absent, leave):
    result = pd.DataFrame()
    for column in df.columns[2:]:
        present_count = df[column].value_counts().get(prenset, 0)
        absent_count = df[column].value_counts().get(absent, 0)
        leave_count = df[column].value_counts().get(leave, 0)
        total_count = present_count + absent_count + leave_count
        present_rate = round((present_count / total_count * 100),2) if total_count > 0 else 0
        result.loc[column, 'Present Count'] = present_count
        result.loc[column, 'Absent Count'] = absent_count
        result.loc[column, 'Leave Count'] = leave_count
        result.loc[column, 'Present Rate (%)'] = present_rate
    return result

In [6]:
def filter_last_x_months(df, x):
    today = datetime.today()
    x_months_ago = today - relativedelta(months=x)
    temp_df = df.copy()
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    return df[temp_df['date'] >= x_months_ago]

In [7]:
resp = getData(url)
df = None
if resp is not None:
    df = getDf_from_google_sheet_json(resp.json())


In [8]:
filtered_df = filter_last_x_months(df,7)
result_df = getDf_present_rate(filtered_df, prenset, absent, leave)
result_df

,Present Count,Absent Count,Leave Count,Present Rate (%)
SzeKi,6.0,0.0,1.0,85.71
Caitlyn,8.0,0.0,0.0,100.00
Yau,5.0,1.0,2.0,62.50
Yannis,8.0,0.0,0.0,100.00
Yoyo,6.0,1.0,1.0,75.00
Heiting,4.0,4.0,0.0,50.00
Chuilam,2.0,4.0,2.0,25.00
Chautung,8.0,0.0,0.0,100.00


In [22]:
result_df.columns = ['Name', 'Present Count', 'Absent Count', 'Leave Count', 'Present Rate (%)']
result_df

,Name,Present Count,Absent Count,Leave Count,Present Rate (%)
0,SzeKi,6.0,0.0,1.0,85.71
1,Caitlyn,8.0,0.0,0.0,100.00
2,Yau,5.0,1.0,2.0,62.50
3,Yannis,8.0,0.0,0.0,100.00
4,Yoyo,6.0,1.0,1.0,75.00
5,Heiting,4.0,4.0,0.0,50.00
6,Chuilam,2.0,4.0,2.0,25.00
7,Chautung,8.0,0.0,0.0,100.00


In [10]:
formated_df = result_df.style.format("{:.2f}".format)

In [11]:
formated_df

,Present Count,Absent Count,Leave Count,Present Rate (%)
SzeKi,6.00,0.00,1.00,85.71
Caitlyn,8.00,0.00,0.00,100.00
Yau,5.00,1.00,2.00,62.50
Yannis,8.00,0.00,0.00,100.00
Yoyo,6.00,1.00,1.00,75.00
Heiting,4.00,4.00,0.00,50.00
Chuilam,2.00,4.00,2.00,25.00
Chautung,8.00,0.00,0.00,100.00
